In [4]:
!pip install selenium
!pip install pandas selenium webdriver-manager
!pip install webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time
import sqlite3
import random 
# import requests

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [2]:
end_date = datetime.now()
start_date = end_date - timedelta(days= 60) # Get 10 years of data 365*10

period1 = int(start_date.timestamp())
period2 = int(end_date.timestamp())


def scrape_yahoo_finance(stock_symbol):
    website_link = f'https://finance.yahoo.com/quote/{stock_symbol}/history/?period1={period1}&period2={period2}'

    # r = requests.get(website_link)

    # if r.status_code == 200:
    #     print('Pass')
    # else:
    #     print('Fail')
    #     raise ExceptionType('Error Message')
    
    # driver = webdriver.Chrome()
    # driver.get(website_link)
    
    # #Wait for the page to load
    # time.sleep(random.uniform(5,10))  # Add a random deloy to avoid bot detection

    # try: 
#     """
#     Data Extraction:
# Uses an XPath to locate the historical data table. Important: Inspect the Yahoo Finance page to ensure this XPath is still correct. Yahoo Finance's HTML structure can change, which would break the scraper.
# table.get_attribute('outerHTML') gets the HTML content of the table.
# pd.read_html(table_html)[0] uses pandas to parse the HTML table into a DataFrame.
# Adds a 'Stock' column to the DataFrame to identify the source of the data.
# Handles a potential "Dividends Only" header row that can appear on some stock's historical data pages.
# """
        # # Extract the historical data table (need verified, asked GPT)
        # table = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table')
        # table_html = table.get_attribute('outerHTML')
        # df = pd.read_html(table_html)[0]

        # # Add stock column
        # sanitized_symbol = stock_symbol.replace('^',"_")
        # df['Stock'] = sanitized_symbol
        
        # #Reorder columns to have 'Stock' as the first column
        # cols = df.columns.tolist()
        # cols = ['Stock'] + cols[:-1] 
        # df = df[cols]
        
        # #Needed?
        # #Handle the 'Dividends Only header row'
        # if 'Dividends Only' in df.columns: 
        #     df = df.drop('Dividends Only, axis = 1')

    driver.get(website_link)

    # Wait for the page to load (adjust time as needed)
    driver.implicitly_wait(10)

    data = []

    try:
        # Locate the table rows
        table_rows = driver.find_elements(By.CSS_SELECTOR, 'table.yf-1jecxey tbody tr')

        # Iterate through each row and extract data
        for row in table_rows:
            # Locate the table cells in each row
            cells = row.find_elements(By.CSS_SELECTOR, 'td.yf-1jecxey')
            
            # Extract the text from each cell
            row_data = [cell.text for cell in cells]
            if row_data:  # Only append if row_data is not empty
                data.append(row_data)

        if data: 
            columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
            # sanitized_symbol = stock_symbol.replace('^','_')
           
            df = pd.DataFrame(data, columns = columns)
            df['Stock_Symbol'] = stock_symbol


    except Exception as e:
        print(f"Error scraping data for {stock_symbol}: {e}")
        data = None # Return None if there was an error
    finally: 
        driver.quit()
    return data
              
        

In [3]:
# stock_symbols = ['AAPL', 'BA', 'T', 'MGM', 'AMZN', 'IBM', 'TSLA', 'GOOG', '^GSPC']
stock_symbols = ['AAPL', '^GSPC'] # test

conn = sqlite3.connect("stocks.db")

for stock in stock_symbols: 
    print(f"Scraping data for {stock}...")
    stock_data = scrape_yahoo_finance(stock)
    sanitized_symbol = stock.replace('^','_')
    if stock_data is not None: 
        # Store the data in the SQLIT database
        stock_data.to_sql(sanitized_symbo, conn, if_exist = "replace", index =True)
        print(f"Data for {stock} successfully stored in database.")
    else: 
        print(f"No data found or error occurred for {stock}.")
    time.sleep(random.uniform(10, 20))
conn.close()
print("Scraping and data storage complete.")

Scraping data for AAPL...


AttributeError: 'list' object has no attribute 'to_sql'

In [ ]:
#Verification
conn = sqlite3.connect("stocks.db")
for symbol in stock_symbols: 
    sanitized_symbol = symbol.replace('^','_')
    read_df = pd.read_sql(f"SELECT * FROM {sanitized_symbol}", conn, index_col = 'Date')
    print(f"Data for {sanitized_symbol}:")
    print(read_df)
conn.close()

In [ ]:
#Code that works
# Reference only
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# import pandas as pd


# # Initialize the Chrome driver
# service = Service(executable_path=ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)

# def scrape_table_data(url):
#     driver.get(url)

#     # Wait for the page to load (adjust time as needed)
#     driver.implicitly_wait(10)

#     data = []

#     try:
#         # Locate the table rows
#         table_rows = driver.find_elements(By.CSS_SELECTOR, 'table.yf-1jecxey tbody tr')

#         # Iterate through each row and extract data
#         for row in table_rows:
#             # Locate the table cells in each row
#             cells = row.find_elements(By.CSS_SELECTOR, 'td.yf-1jecxey')
            
#             # Extract the text from each cell
#             row_data = [cell.text for cell in cells]
#             data.append(row_data)

#     except Exception as e:
#         print(f"Error scraping data: {e}")
#         data = None

#     finally:
#         return data

# # Example URL (replace with your actual URL)
# url = 'https://finance.yahoo.com/quote/AAPL/history?period1=1472668800&period2=1693084800'

# table_data = scrape_table_data(url)

# if table_data:
#     # Define the column names
#     columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

#     # Create a Pandas DataFrame
#     df = pd.DataFrame(table_data, columns=columns)
#     print(df)
# else:
#     print("No data scraped.")

# driver.quit()